In [44]:
#import libraries
import teradata as td
import pandas as pd
from getpass import getpass, getuser

In [45]:
#log in to Teradata
udaExec = td.UdaExec(appName="Pfizer_KOL_CMS_App", version="1.0",
        logConsole=False)

uname = getuser()
print(uname)
pword = getpass("Teradata Password: ")
 
session = udaExec.connect(method="odbc", system="amrndhp3",
        username=uname, password=pword);

WEITEM
Teradata Password: ········


In [46]:
%%time

# get CMS data
listed_data = []
data = session.execute("SELECT * FROM HVAD_TEAM_T.mw_cms_kol_all_2k17")
for row in data:
    listed_data.append(row[0:])

Wall time: 15.1 s


In [38]:
# get column names
col_names = []
col_names_ter = session.execute("SELECT ColumnName FROM dbc.ColumnsV WHERE DatabaseName='HVAD_TEAM_T' AND TableName='mw_cms_kol_all_2k17' ORDER BY ColumnId")
for item in col_names_ter:
    col_names.append(item[0])
col_names

['record_id',
 'phys_full_name',
 'assoc_type',
 'product',
 'payment_date',
 'amount',
 'num_payments',
 'payment_nature',
 'Payment_Year',
 'submitting_company',
 'paying_company',
 'consolidating_company']

In [40]:
# make dataframe
df = pd.DataFrame(data=listed_data, columns = col_names)
df.head()

,record_id,phys_full_name,assoc_type,product,payment_date,amount,num_payments,payment_nature,Payment_Year,submitting_company,paying_company,consolidating_company
0,389957222,"Soloman, Nehad",recipient,PROLIA,2016-10-06,123.19,1,Food and Beverage,2016,Amgen Inc.,Amgen Inc.,Amgen Inc.
1,31093345,"Strand, Vibeke",recipient,None,2013-11-06,65.07,1,Travel and Lodging,2013,"AbbVie, Inc.","AbbVie, Inc.","AbbVie, Inc."
2,309188896,"Aelion, Jacob",recipient,HUMIRA,2015-02-25,12.95,1,Food and Beverage,2015,"AbbVie, Inc.","AbbVie, Inc.","AbbVie, Inc."
3,359329330,"Lambert, Rhonda E",recipient,KRYSTEXXA,2016-08-12,43.14,1,Food and Beverage,2016,Horizon Pharma plc,Horizon Pharma plc,Horizon Pharma plc
4,388751724,"Huffstutter, Joseph E",recipient,ENBREL,2016-11-11,12.40,1,Food and Beverage,2016,Amgen Inc.,Amgen Inc.,Amgen Inc.


In [43]:
from sklearn import ensemble

ensemble.IsolationForest

119402